In [2]:
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load dataset
file_path = "data.csv"
df = pd.read_csv(file_path)

# Encode labels
label_encoder = LabelEncoder()
df["Sentiment"] = label_encoder.fit_transform(df["Sentiment"])

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Sentence"].tolist(), df["Sentiment"].tolist(), test_size=0.2, random_state=42
)

# Load FinBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")

# Tokenize dataset
def tokenize(texts, tokenizer, max_length=128):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

train_encodings = tokenize(train_texts, tokenizer)
test_encodings = tokenize(test_texts, tokenizer)

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

# Load pre-trained FinBERT model
model = TFAutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train model
model.fit(
    x={"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    y=train_labels,
    validation_data=(
        {"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]},
        test_labels
    ),
    epochs=3,
    batch_size=16
)

# Save model
model.save_pretrained("./finbert_finetuned")
tokenizer.save_pretrained("./finbert_finetuned")

print("Fine-tuning complete. Model saved.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/439M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Epoch 1/3
293/293 [==============================] - 175s 441ms/step - loss: 0.8638 - sparse_categorical_accuracy: 0.6726 - val_loss: 0.5037 - val_sparse_categorical_accuracy: 0.7784
Epoch 2/3
293/293 [==============================] - 129s 439ms/step - loss: 0.4104 - sparse_categorical_accuracy: 0.8209 - val_loss: 0.4962 - val_sparse_categorical_accuracy: 0.7725
Epoch 3/3
293/293 [==============================] - 129s 440ms/step - loss: 0.2666 - sparse_categorical_accuracy: 0.8742 - val_loss: 0.5682 - val_sparse_categorical_accuracy: 0.8067
Fine-tuning complete. Model saved.
